# Classification Downstream Task 

In [ ]:
!pip install ssl_remote_sensing@git+https://github.com/AlexanderLontke/ssl-remote-sensing.git@simclr/bigearthnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AlexanderLontke/ssl-remote-sensing.git (to revision simclr/bigearthnet) to /tmp/pip-install-12epou67/ssl-remote-sensing_0b82401ac3a1458dbf02839caa7eb621
  Running command git clone -q https://github.com/AlexanderLontke/ssl-remote-sensing.git /tmp/pip-install-12epou67/ssl-remote-sensing_0b82401ac3a1458dbf02839caa7eb621
  Running command git checkout -b simclr/bigearthnet --track origin/simclr/bigearthnet
  Switched to a new branch 'simclr/bigearthnet'
  Branch 'simclr/bigearthnet' set up to track remote branch 'simclr/bigearthnet' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [ ]:
# Log in to your W&B account
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alontke. Use `wandb login --relogin` to force relogin


True

In [ ]:
from datetime import datetime
import torch
import os
import numpy as np
from tqdm import tqdm

from torch import nn
import torchvision.transforms as T
from sklearn.metrics import classification_report

from ssl_remote_sensing.downstream_tasks.classification.model import (
    DownstreamClassificationNet,
)
from ssl_remote_sensing.constants import RANDOM_INITIALIZATION
from ssl_remote_sensing.pretext_tasks.utils import (
    load_encoder_checkpoint_from_pretext_model,
)

from ssl_remote_sensing.data.eurosat.eurosat_dataset import get_eurosat_normalizer
from ssl_remote_sensing.data.eurosat.get_12_band_eurosat import get_eurosat_dataloader

/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.8/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveTask] = FeatureMapContr

## Dataset Loading ##

In [ ]:
class RunConfig:
    def __init__(self, freeze_encoder, use_linear_head):
        self.num_epochs = 30  # number of training epochs
        self.seed = 1234  # randomness seed
        self.save = "./saved_models/"  # save checkpoint
        self.batch_size = 128
        self.learning_rate = 1e-3
        self.embedding_size = 128  # papers value is 128
        self.test_split_ratio = 0.2
        self.checkpoint_name = None
        self.freeze_encoder = freeze_encoder
        self.use_linear_head = use_linear_head

bool_vals = [False, True]
configs = []
for fe in bool_vals:
  for lh in [False]: #bool_vals:
    configs += [RunConfig(freeze_encoder=fe, use_linear_head=lh)]

In [ ]:
[conf.__dict__ for conf in configs]

[{'num_epochs': 30,
  'seed': 1234,
  'save': './saved_models/',
  'batch_size': 128,
  'learning_rate': 0.001,
  'embedding_size': 128,
  'test_split_ratio': 0.2,
  'checkpoint_name': None,
  'freeze_encoder': False,
  'use_linear_head': False},
 {'num_epochs': 30,
  'seed': 1234,
  'save': './saved_models/',
  'batch_size': 128,
  'learning_rate': 0.001,
  'embedding_size': 128,
  'test_split_ratio': 0.2,
  'checkpoint_name': None,
  'freeze_encoder': True,
  'use_linear_head': False}]

In [ ]:
#!mkdir /content/eurosat
#!curl https://madm.dfki.de/files/sentinel/EuroSATallBands.zip --output /content/eurosat/EuroSATallBands.zip
#!unzip /content/eurosat/EuroSATallBands.zip -d /content/eurosat

In [ ]:
# Setup data loading
# eurosat_normalizer = get_eurosat_normalizer()
#train_dl, test_dl = get_eurosat_dataloader(
#    root="/content/eurosat/ds/images/remote_sensing/otherDatasets/sentinel_2/tif",
#    transform=T.Compose([eurosat_normalizer]),
#    batchsize=configs[0].batch_size,
#    numworkers=os.cpu_count(),
#    split=(configs[0].test_split_ratio==0.2),
#    in_memory=True,
#)


## Model Training ##


In [ ]:
# First of all, let's verify if a GPU is available on our compute machine. If not, the cpu will be used instead.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used: {}".format(device))

# define the optimization criterion / loss function
loss_criterion = nn.CrossEntropyLoss().to(device)

Device used: cuda:0


## Setup Checkpoint Loading ##

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
g_drive_path = "/content/drive/MyDrive/deep_learning_checkpoints"
check_point_paths = os.listdir(g_drive_path)
check_point_paths += [RANDOM_INITIALIZATION]
check_point_paths = [g_drive_path + "/" + x for x in check_point_paths]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
check_point_paths

['/content/drive/MyDrive/deep_learning_checkpoints/SimCLR_BEN_ResNet18_adam_bs500-v1.ckpt',
 '/content/drive/MyDrive/deep_learning_checkpoints/vae_bigearthnet.ckpt',
 '/content/drive/MyDrive/deep_learning_checkpoints/bigan_encoder_allChannels_10_epochs.ckpt',
 '/content/drive/MyDrive/deep_learning_checkpoints/random']

In [ ]:
eurosat_normalizer = get_eurosat_normalizer()
# Data samples loop
for max_samples in [216,  2160, 10800, 21600]:
    train_dl, test_dl = get_eurosat_dataloader(
        root="/content/eurosat/ds/images/remote_sensing/otherDatasets/sentinel_2/tif",
        transform=T.Compose([eurosat_normalizer]),
        batchsize=configs[0].batch_size,
        numworkers=os.cpu_count(),
        split=(configs[0].test_split_ratio==0.2),
        in_memory=True,
        max_samples=max_samples
    )
    # Configs loop
    for config in configs:
        # Pre-text tasks loop
        for filename in check_point_paths:
            # Update checkpoint name
            config.checkpoint_name = filename
            # Load Encoder from different pre-text architectures
            encoder = load_encoder_checkpoint_from_pretext_model(
                path_to_checkpoint=filename,
            )
            wandb.init(
                project="ssl-remote-sensing-classification-label-fraction-experiment",
                name=filename.split("/")[-1] \
                + f"_lr_{config.learning_rate}" \
                + f"_frozen_{config.freeze_encoder}" \
                + f"_linear_{config.use_linear_head}" \
                + f"_dataset_size_{max_samples}",
                config=config.__dict__,
            )
            # Model Setup
            if "bigan" in config.checkpoint_name.lower():
                model = DownstreamClassificationNet(
                    input_dim=100,
                    encoder=encoder,
                    gan_encoder = True,
                    freeze_encoder=config.freeze_encoder,
                    use_linear_head=config.use_linear_head
                ).to(device)
            else:
                model = DownstreamClassificationNet(
                    input_dim=512, 
                    encoder=encoder,
                    freeze_encoder=config.freeze_encoder,
                    use_linear_head=config.use_linear_head
                ).to(device)
            # define learning rate and optimization strategy
            optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
            model.train()
            train_epoch_loss = np.NaN
            validation_epoch_loss = np.NaN

            with tqdm(range(config.num_epochs)) as tq:
                for epoch in tq:
                    # print epoch loss
                    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
                    tq.desc = f"[{now}] epoch: {epoch+1} train-loss: {train_epoch_loss} validation-loss: {validation_epoch_loss}"
                    # init collection of mini-batch losses
                    train_mini_batch_losses = []

                    # iterate over all-mini batches
                    for i, (labels, images) in enumerate(train_dl):
                        # push mini-batch data to computation device
                        images = images.to(device)
                        labels = labels.to(device)

                        # forward + backward + optimize
                        optimizer.zero_grad()
                        out = model(images)
                        loss = loss_criterion(out, labels)
                        loss.backward()
                        optimizer.step()

                        # collect mini-batch reconstruction loss
                        train_mini_batch_losses.append(loss.data.item())
                        wandb.log(
                            {
                                "step/training_loss": loss.data.item(),
                            }
                        )

                    # determine mean min-batch loss of epoch
                    train_epoch_loss = np.mean(train_mini_batch_losses)

                    # Specify you are in evaluation mode
                    y_pred = []
                    y_true = []
                    model.eval()
                    with torch.no_grad():
                        validation_mini_batch_losses = []
                        for (labels, images) in test_dl:
                            images = images.to(device)
                            labels = labels.to(device)
                            # calculate outputs by running images through the network
                            outputs = model(images)
                            _, predicted = torch.max(outputs, 1)
                            y_pred.extend(predicted.cpu().numpy())  # Save Prediction
                            y_true.extend(labels.cpu().numpy())  # Save Truth
                            # the class with the highest energy is what we choose as prediction
                            validation_epoch_loss = loss_criterion(outputs, labels)
                            # collect mini-batch reconstruction loss
                            validation_mini_batch_losses.append(
                                validation_epoch_loss.data.item()
                            )
                        validation_epoch_loss = np.mean(validation_mini_batch_losses)

                    wandb.log(
                        {
                            "epoch/training_loss": train_epoch_loss,
                            "epoch/validation_loss": validation_epoch_loss,
                            "epoch/classification_report": classification_report(
                                y_true, y_pred, output_dict=True
                            )
                        }
                    )

Loading Images: 100%|██████████| 10/10 [01:18<00:00,  7.87s/it]


[LOG] Total number of images: 27000
[LOG] Batch size is 128
[LOG] Total images in the train set is: 216
[LOG] Total number of batches in the trainloader: 2
[LOG] Total number of batches in the valloader: 43


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[20221228-09:36:10] epoch: 1 

epoch/training_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/validation_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step/training_loss,▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/training_loss,2.27132
epoch/validation_loss,2.32841
step/training_loss,2.27499


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[20221228-09:39:13] epoch: 1 

...

[20221229-00:50:17] epoch: 30 train-loss: 0.42784767337804713 validation-loss: 0.6133738111617953: 100%|██████████| 30/30 [1:34:36<00:00, 189.23s/it]


epoch/training_loss,█▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch/validation_loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▂▂▁▁▁▂▁▁▁▁
step/training_loss,█▅▆▅▅▃▄▅▃▄▅▃▄▂▃▄▄▃▂▂▃▃▃▂▂▂▂▂▃▂▂▃▂▂▂▃▁▂▂▂
epoch/training_loss,0.40492
epoch/validation_loss,0.62666
step/training_loss,0.46761


  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[20221229-02:25:29] epoch: 30

epoch/training_loss,█▆▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/validation_loss,█▆▅▅▄▃▃▃▃▂▄▃▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁
step/training_loss,█▅▆▅▄▃▄▃▄▅▄▄▄▄▃▃▃▃▃▃▄▄▃▃▃▂▂▄▂▂▃▃▂▄▃▁▂▂▁▂
epoch/training_loss,1.00416
epoch/validation_loss,1.14308
step/training_loss,0.99266


[20221229-04:00:16] epoch: 30 train-loss: 0.2508288668633918 validation-loss: 1.0359439614207246: 100%|██████████| 30/30 [1:34:37<00:00, 189.24s/it]


In [ ]:
wandb.finish()

epoch/training_loss,█▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
epoch/validation_loss,█▅▄▃▂▃▄▂▁▁▁▂▁▂▂▂▁▂▂▃▄▄▄▃▄▅▆▇▇█
step/training_loss,█▆▇▅▄▄▄▄▄▅▄▄▃▄▄▄▂▄▃▃▃▃▂▃▂▂▃▃▂▂▁▂▂▁▂▁▂▂▁▁
epoch/training_loss,0.23022
epoch/validation_loss,1.08437
step/training_loss,0.16106
